In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 3

train_dir = '/Users/nathan/Desktop/Code/Tomato-disease/training/PlantVillage/train'
test_dir = '/Users/nathan/Desktop/Code/Tomato-disease/training/PlantVillage/validation'

# U-Net model
def create_unet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Contracting path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # ...

    # Expanding path
    up1 = UpSampling2D(size=(2, 2))(conv2)
    up1 = Conv2D(64, 2, activation='relu', padding='same')(up1)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)

    up2 = UpSampling2D(size=(2, 2))(conv6)
    up2 = Conv2D(64, 2, activation='relu', padding='same')(up2)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

    # Output
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(conv7)

    return Model(inputs, outputs)

# Load and preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Create the U-Net model
unet_model = create_unet_model(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES)

# Create the ResNet50 model
resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(*IMAGE_SIZE, 3))

# Freeze the layers of the ResNet50 model
for layer in resnet_model.layers:
    layer.trainable = False

# Get the output of the U-Net model
unet_output = unet_model.output

# Connect the output of the U-Net model to the input of the ResNet50 model
resnet_input = resnet_model(unet_output)

# Add a flatten layer and a dense layer on top of the ResNet50 model
x = Flatten()(resnet_input)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the combined model
combined_model = Model(inputs=unet_model.input, outputs=predictions)

# Compile the combined model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the combined model
history = combined_model.fit(
    train_data,
    epochs=EPOCHS
)

# Evaluate the combined model
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = combined_model.evaluate(test_data)

# Plot the training accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Training Accuracy and Loss')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')
plt.show()


Found 3599 images belonging to 3 classes.
Epoch 1/10
113/113 [==============================] - 8216s 72s/step - loss: 2.9971 - accuracy: 0.4251
Epoch 2/10
 71/113 [=================>............] - ETA: 52:52 - loss: 0.9451 - accuracy: 0.5845

In [ ]:
from sklearn.metrics import roc_curve, confusion_matrix, precision_recall_curve
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_score, recall_score


# Get the predicted probabilities for each class
y_pred_proba = combined_model.predict(test_data)
y_pred_labels = np.argmax(y_pred_proba, axis=1)

# Get the true labels of the test data
y_true_labels = test_data.classes

# Calculate the validation loss
val_loss = test_loss

# Print the validation loss
print("Validation Loss:", val_loss)

y_true_one_hot = label_binarize(y_true_labels, classes=range(NUM_CLASSES))

# Calculate and plot ROC curves for each class
for class_index in range(NUM_CLASSES):
    fpr, tpr, _ = roc_curve(y_true_one_hot[:, class_index], y_pred_proba[:, class_index])
    plt.plot(fpr, tpr, label=f'Class {class_index}')

plt.title('ROC Curves for Each Class')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

# Calculate class percentages
class_counts = np.sum(y_true_one_hot, axis=0)
class_percentages = class_counts / np.sum(class_counts)
class_labels = [f'Class {i} ({percentage*100:.2f}%)' for i, percentage in enumerate(class_percentages)]

# Show class percentages in the graph
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.gca().set_aspect('equal', adjustable='box')
plt.text(0.5, 0.05, '\n'.join(class_labels), horizontalalignment='center', verticalalignment='center',
         transform=plt.gca().transAxes, bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8))

plt.show()
# Calculate the confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

# Calculate and plot precision-recall curves for each class
for class_index in range(NUM_CLASSES):
    y_true_binary = y_true_one_hot[:, class_index]
    y_pred_binary = y_pred_proba[:, class_index]
    precision = precision_score(y_true_binary, y_pred_binary)
    recall = recall_score(y_true_binary, y_pred_binary)
    plt.plot(recall, precision, marker='o', markersize=5, label=f'Class {class_index}')

plt.title('Precision-Recall Curves for Each Class')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()